In [8]:
import numpy as np
from sklearn import datasets
from sklearn import tree
from numpy.linalg import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

import bbreg
from common import *

In [ ]:
# Define a learning algorithm using a decision tree with correct interface
def learning_alg(X, Y):
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X, Y)
    return model.predict

In [40]:

def test_regularization_method(regularization_method, eval_criterion, verbose):
    try:
        # Generate a train/test split
        X, Y = make_regression(n_samples=50, n_features=100, noise=100, random_state=42)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

        # Test black_box_regress
        predictor = bbreg.black_box_regress(
            learning_alg=learning_alg,
            X=X_train,  
            Y=Y_train,
            regularization_method=regularization_method,
            eval_criterion=eval_criterion,
            K=5,
            M=5,
            verbose=verbose
        )
        Yhat = predictor(X_test)  # Predict on the test set

        # Compute loss
        if eval_criterion == EvalCriterion.MSE:
            eval_value = mse(Y_test, Yhat)
        elif eval_criterion == EvalCriterion.MAD:
            eval_value = mad(Y_test, Yhat)


        assert isinstance(eval_value, float), f"{eval_criterion} value should be a float"
        print(f"Test Passed - Regularization: {regularization_method}, Eval Criterion: {eval_criterion}, Verbose: {verbose}")
    except AssertionError as e:
        print(f"Assertion Error: {e}")
    except Exception as e:
        print(f"Error during test: {e}")




In [41]:
def test_input_validation():
    # Mock learning algorithm for testing
    X_correct = np.random.randn(50, 10)
    Y_correct = np.random.randn(50)
    X_wrong_dim = np.random.randn(50)  # Incorrect dimension
    Y_wrong_dim = np.random.randn(50, 10)  # Incorrect dimension

    # Test with correct dimensions
    try:
        bbreg.black_box_regress(
            learning_alg=learning_alg,
            X=X_correct,
            Y=Y_correct,
            regularization_method="Dropout",  # Assuming string identifiers for simplicity
            eval_criterion="MSE",
            K=5,
            M=5,
            verbose=False
        )
        print("Test 1.1 Passed: Correct input dimensions.")
    except Exception as e:
        print(f"Test 1.1 Failed: {e}")

    # Test with incorrect dimensions for X
    try:
        bbreg.black_box_regress(
            learning_alg=learning_alg,
            X=X_wrong_dim,  # Wrong dimension
            Y=Y_correct,
            regularization_method="Dropout",
            eval_criterion="MSE",
            K=5,
            M=5,
            verbose=False
        )
        print("Test 1.2 Failed: No error raised for incorrect X dimension.")
    except ValueError:
        print("Test 1.2 Passed: Correctly identified incorrect X dimension.")

    # Test with incorrect dimensions for Y
    try:
        bbreg.black_box_regress(
            learning_alg=learning_alg,
            X=X_correct,
            Y=Y_wrong_dim,  # Wrong dimension
            regularization_method="Dropout",
            eval_criterion="MSE",
            K=5,
            M=5,
            verbose=False
        )
        print("Test 1.3 Failed: No error raised for incorrect Y dimension.")
    except ValueError:
        print("Test 1.3 Passed: Correctly identified incorrect Y dimension.")

    # Missing M for Dropout and Noise Addition
    try:
        bbreg.black_box_regress(
            learning_alg=learning_alg,
            X=X_correct,
            Y=Y_correct,
            regularization_method="Dropout",
            eval_criterion="MSE",
            K=5,
            M=None,  # Missing M
            verbose=False
        )
        print("Test 1.4 Failed: No error raised for missing M parameter.")
    except ValueError:
        print("Test 1.4 Passed: Correctly identified missing M parameter.")

In [43]:
#Test correct and incorrect input dimensions of arrays 
test_input_validation()

# Run tests with different (correct) combinations of regularization methods, evaluation criteria, and verbosity
for reg_method in ['Dropout', 'NoiseAddition']:
    for eval_crit in ['MSE', 'MAD']:
        for verb in [False, True]:
            test_regularization_method(reg_method, eval_crit, verb)


Test 1.1 Passed: Correct input dimensions.
Test 1.2 Passed: Correctly identified incorrect X dimension.
Test 1.3 Passed: Correctly identified incorrect Y dimension.
Test 1.4 Passed: Correctly identified missing M parameter.
Test Passed - Regularization: Dropout, Eval Criterion: MSE, Verbose: False
Optimal Dropout Probability: 0.0
Test Passed - Regularization: Dropout, Eval Criterion: MSE, Verbose: True
Test Passed - Regularization: Dropout, Eval Criterion: MAD, Verbose: False
Optimal Dropout Probability: 0.0
Test Passed - Regularization: Dropout, Eval Criterion: MAD, Verbose: True
Test Passed - Regularization: NoiseAddition, Eval Criterion: MSE, Verbose: False
Optimal Distribution: Distribution.normal Optimal Standard Deviation: 2.1279166666666667
Test Passed - Regularization: NoiseAddition, Eval Criterion: MSE, Verbose: True
Test Passed - Regularization: NoiseAddition, Eval Criterion: MAD, Verbose: False
Optimal Distribution: Distribution.logistic Optimal Standard Deviation: 1.5050000